## Logistic Regression (LR) on UAE Cancer Patient Dataset

**Course:** CDS 2413 — Programming for Data Science  
**CRN:** 14350  
**Project:** Cancer Outcome Classification (UAE Synthetic Records)  
**Group:** 3  
**Members:** Aisha Alshamsi (H00535685), Sara Almulla (H00532724)  
**Dataset Size:** 10,000 records

## Importing the libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Importing the dataset

In [6]:
df = pd.read_csv("cancer_dataset_uae.csv")

df = df.dropna(subset=["Outcome"])
df["Outcome_binary"] = (df["Outcome"] == "Recovered").astype(int)

feature_cols = [
    "Age","Gender","Emirate","Cancer_Type","Cancer_Stage",
    "Smoking_Status","Comorbidities","Weight","Height"
]
data = df[feature_cols + ["Outcome_binary"]].dropna()

X = pd.get_dummies(data[feature_cols], drop_first=True)
y = data["Outcome_binary"]

print("X shape:", X.shape)
print("y shape:", y.shape)
display(X.head())
print(y.value_counts())



X shape: (5953, 25)
y shape: (5953,)


Age  Weight  Height  Gender_Male  Gender_Other  Emirate_Ajman  \
5    41      59     176        False         False          False   
6    20      66     155         True         False          False   
10   47      48     173         True         False          False   
11   55      71     160        False         False          False   
13   81      84     168         True         False          False   

    Emirate_Dubai  Emirate_Fujairah  Emirate_Ras Al Khaimah  Emirate_Sharjah  \
5           False             False                   False            False   
6           False             False                    True            False   
10          False             False                   False            False   
11          False             False                   False            False   
13          False             False                    True            False   

    ...  Cancer_Type_Ovarian  Cancer_Type_Pancreatic  Cancer_Type_Prostate  \
5   ...                False                   False                 False   
6   ...                 True                   False                 False   
10  ...                False                   False                  True   
11  ...                False                   False                 False   
13  ...                False                   False                 False   

    Cancer_Stage_II  Cancer_Stage_III  Cancer_Stage_IV  \
5             False             False            False   
6             False             False             True   
10             True             False            False   
11            False              True            False   
13            False             False             True   

    Smoking_Status_Non-Smoker  Smoking_Status_Smoker  Comorbidities_Diabetes  \
5                       False                   True                   False   
6                        True                  False                   False   
10                      False                  False                   False   
11                      False                   True                    True   
13                       True                  False                   False   

    Comorbidities_Hypertension  
5                         True  
6                         True  
10                       False  
11                       False  
13                        True  

[5 rows x 25 columns]

Outcome_binary
1    2980
0    2973
Name: count, dtype: int64


## Encoding Categorical Variable

In [7]:

X = X.astype(int)  
y = y.astype(int)

print("X shape:", X.shape, "| y shape:", y.shape)
print("Numeric cols in X:", (X.dtypes == "int32").sum() + (X.dtypes == "int64").sum())


X shape: (5953, 25) | y shape: (5953,)
Numeric cols in X: 25


## Splitting the dataset into the Training set and Test set

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y 
)


## Feature Scaling

In [9]:

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)   
X_test  = sc.transform(X_test)        

print(X_train.shape, X_test.shape)


(4762, 25) (1191, 25)


## Training the Logistic Regression model on the Training set

In [10]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state = 0, max_iter=1000)

classifier.fit(X_train, y_train)


LogisticRegression(max_iter=1000, random_state=0)

## Making the Confusion Matrix

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

y_pred = classifier.predict(X_test)


cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
acc = accuracy_score(y_test, y_pred)

print("Confusion Matrix:\n", cm)
print("Accuracy:", round(acc, 3))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["NotRecovered","Recovered"]))


Confusion Matrix:
 [[300 295]
 [293 303]]
Accuracy: 0.506

Classification Report:
              precision    recall  f1-score   support

NotRecovered       0.51      0.50      0.51       595
   Recovered       0.51      0.51      0.51       596

    accuracy                           0.51      1191
   macro avg       0.51      0.51      0.51      1191
weighted avg       0.51      0.51      0.51      1191



In [13]:
TN, FP, FN, TP = cm.ravel()
print(f"TN={TN}, FP={FP}, FN={FN}, TP={TP}")


TN=300, FP=295, FN=293, TP=303


## Predicting the value

In [14]:

print(classifier.predict(X_test[:1]))          
print(classifier.predict_proba(X_test[:1])[:,1])


[1]
[0.54358222]


## Predicting the value for another patient.

In [15]:

def predict_patient(sample_dict):
    new_df = pd.DataFrame([sample_dict])

    newX = pd.get_dummies(new_df, drop_first=True).reindex(columns=X.columns, fill_value=0)

    newX_scaled = sc.transform(newX)

    pred = classifier.predict(newX_scaled)[0]
    proba = classifier.predict_proba(newX_scaled)[0, 1]
    return int(pred), float(proba)

pred, proba = predict_patient({
    "Age": 55,
    "Gender": "Male",
    "Emirate": "Dubai",
    "Cancer_Type": "Breast",
    "Cancer_Stage": "II",
    "Smoking_Status": "Never",
    "Comorbidities": "None",
    "Weight": 72,
    "Height": 170
})
print("Predicted label:", pred)                
print("Prob(Recovered):", round(proba, 3))


Predicted label: 0
Prob(Recovered): 0.498
